## Home Credit competition

- 他のチームの結果を参考に, LightGBMの使い方, pythonコードの作成方法についてまとめる
- コメント文など適宜追加する

## References

- [Kaggle:How to LightGBM with lightgbm.cv](https://www.kaggle.com/shep312/lightgbm-harder-better-slower/code)
- [Python API](https://lightgbm.readthedocs.io/en/latest/Python-API.html)
- [roc auc score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html0)
- [1st place solution](https://www.kaggle.com/c/home-credit-default-risk/discussion/64821)
- [2nd place solution](https://www.kaggle.com/c/home-credit-default-risk/discussion/64722)

## Winner session summary

Important thing is

- Good set of smart features
- Diverse set of base algorithms



## Import modules

In [9]:
# import module
import gc, os, sys, warnings,time
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from contextlib import contextmanager
from multiprocessing import cpu_count, Pool
from collections import defaultdict
from glob import glob
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

# set path
sys.path.append("/Users/takuto/Desktop/Home_Credit_Kaggle/py/") # for macbook
sys.path.append("/Users/takutokotsubo/Desktop/Home_Credit_Kaggle/py/") # for imac

## function

- @contextmanager: with文を用いて簡単に実行時間を計測できる関数を作成できる
- one-hot: デジタル回路において, 1つだけ"1"の値をとり, 他の全ての値が"0"であるようなビット列, ダミー変数化では0,1の値を用いて区別するので少し意味合いが違うことに注意
- gc.collect(): del -> gc.collect()でメモリを開放でできる

In [10]:
# Execution time measuring
@contextmanager
def timer(title):
    t0 = time.time()
    yield print("{} - done in {:.0f}s".format(title, time.time() - t0))
    
# One-hot encoding for categorical columns with get_dummies
# not using
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == "object"]
    df = pd.get_dummies(df, columns = categorical_columns, dummy_na = nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df,new_columns

# preprocess all data
# not using
def read_all_data(num_rows = None, nan_as_category = False):
    df = pd.read_csv("../input/csv_imp1/all_data_train.csv", nrows = num_rows)
    test = pd.read_csv("../input/csv_imp1/all_data_test.csv", nrows = num_rows)
    df = df.append(test).reset_index()
    # Categorical features with Binary encode (0 or 1; two categories)
    # for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    #    df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    del test
    gc.collect()
    return df

# LightGBM with cv, not completed !!!!!!!!!!!
def lightgbm_cv(df, num_folds, NFOLD, LOOP, debug = False):
    # parameters
    param = {
         'objective': 'binary',
         'metric': 'auc',
         'learning_rate': 0.01,
         'max_depth': 6,
         'num_leaves': 63,
         'max_bin': 255,
         'min_child_weight': 10,
         'min_data_in_leaf': 150,
         'reg_lambda': 0.5,  # L2 regularization term on weights.
         'reg_alpha': 0.5,  # L1 regularization term on weights.
         'colsample_bytree': 0.9,
         'subsample': 0.9,
         'nthread': cpu_count(),
         'bagging_freq': 1,
         'verbose':-1,
         }
    train_df = pd.read_csv("../input/csv_imp1/all_data_train.csv", nrows = num_rows)
    test_df = pd.read_csv("../input/csv_imp1/all_data_test.csv", nrows = num_rows)
    print("Starting Light GBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df; gc.collect()
    # non list
    feats = [f for f in train_df.columns if f not in ["TARGET","SK_ID_CURR","SK_ID_BUREAU","SK_ID_PREV","index"]]
    train_x, train_y, test_X = train_df[feats], train_df["TARGET"], test_df[feats]      
    dtrain = lgb.Dataset(train_x,train_y)
    gc.collect()
    # result box
    model_all = []
    y_pred = pd.Series(0, index=train_y.index)
    
    # training with cv
    for i in range(LOOP):
        gc.collect()
        ret, models = lgb.cv(param, dtrain, 9999, nfold=NFOLD,
                             early_stopping_rounds=100, verbose_eval=50, seed=i)
        model_all += models
        
# Display/plot feature importance
# maybe not need 
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    # plt.savefig('lgbm_importances01.png')

# send message to line
def send_line(message):
    line_notify_token = '5p5sPTY7PrQaB8Wnwp6aadfiqC8m2zh6Q8llrfNisGT'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    payload = {'message': message}
    headers = {'Authorization': 'Bearer ' + line_notify_token}
    requests.post(line_notify_api, data=payload, headers=headers)

# main, not completed !!!!!!!!!!
def main(debug = False):
    num_rows = 100000 if debug else None # for debug
    df = read_all_data(num_rows)
    with timer("Run LightGBM with kfold"):
        feat_importance = kfold_lightgbm(df, num_folds= 5, stratified= False, debug= debug)

## Light GBM cross validation

- メモ形式
- 完成したら関数化
- 重要な特徴量の抽出も可能にする

In [11]:
# function para
LOOP = 1
NFOLD = 5 # good set: 7
num_rows = None # good set: None

In [12]:
# input data
## imp_csv0
train_df = pd.read_csv("../input/csv_imp0/all_data_train.csv", nrows = num_rows)
test_df = pd.read_csv("../input/csv_imp0/all_data_test.csv", nrows = num_rows)
## imp_csv1
# train_df = pd.read_csv("../input/csv_imp1/all_data_train.csv", nrows = num_rows)
# test_df = pd.read_csv("../input/csv_imp1/all_data_test.csv", nrows = num_rows)

# select using columns
## all colmuns
feats = [f for f in train_df.columns if f not in ["target_app","sk_id_curr","sk_id_bureau","sk_id_prev"]]
## best columns
# feats = pd.read_table("../data/best_para.tsv")
# feats = feats["Feature"].values.tolist()

In [21]:
# parameters
param = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'max_depth': 6,
    'num_leaves': 63,
    'max_bin': 255,
    
    'min_child_weight': 10,
    'min_data_in_leaf': 150,
    'reg_lambda': 0.5,  # L2 regularization term on weights.
    'reg_alpha': 0.5,  # L1 regularization term on weights.
         
    'colsample_bytree': 0.9,
    'subsample': 0.9,
    'nthread': cpu_count(),
    'bagging_freq': 1,
    'verbose':-1,
}
# make data set
X_train, y_train, X_test = train_df[feats], train_df["target_app"], test_df[feats]      
dtrain = lgb.Dataset(X_train,y_train,free_raw_data=False)
gc.collect()
# result box
model_all = []
y_pred = pd.Series(0, index=y_train.index)

In [ ]:
# training with cv
for i in range(LOOP):
    gc.collect()
    # cross validation
    print("Starting Light GBM {}-fold cross varidation. Train shape: {}, test shape: {}"
          .format(NFOLD, train_df.shape, test_df.shape))
    models_cv = lgb.cv(param, dtrain, 9999, nfold=NFOLD,
                       early_stopping_rounds=100, verbose_eval=50, seed=i)
    print("best n_estimator", len(models_cv["auc-mean"])) # best iteration
    print("best cv score", models_cv["auc-mean"][-1]) # best cv score
    # execute model
    print("Starting Light GBM Training. Train shape: {}, test shape: {}, best iter: {}"
          .format(train_df.shape, test_df.shape, len(models_cv["auc-mean"])))
    models_train = lgb.train(param,dtrain,num_boost_round=len(models_cv["auc-mean"]))
    # predict target (model.best_iteration??)
    print("Starting Light GBM Predict. Train shape: {}, test shape: {}, num iter: {}"
          .format(train_df.shape, test_df.shape, model.best_iteration))
    pred_train = models_train.predict(train_x)
    
    # roc_auc score
    auc_mean = roc_auc_score(train_y,pred_train)
    print(f"CV auc-mean(loop {i}): {auc_mean} {models['auc-mean'][-1]}")
    
    #model_all += models

- さらに細かいメモ

In [10]:
# cross validation
print("Starting Light GBM {}-fold cross varidation. Train shape: {}, test shape: {}"
      .format(NFOLD, train_df.shape, test_df.shape))
models_cv = lgb.cv(param, dtrain, 9999, nfold=NFOLD,
                   early_stopping_rounds=100, verbose_eval=50, seed=i)
print("best n_estimator", len(models_cv["auc-mean"])) # best iteration
print("best cv score", models_cv["auc-mean"][-1]) # best cv score

Starting Light GBM 5-fold cross varidation. Train shape: (307511, 1309), test shape: (48744, 1309)
[50]	cv_agg's auc: 0.743005 + 0.00141292
[100]	cv_agg's auc: 0.750913 + 0.00141804
[150]	cv_agg's auc: 0.757032 + 0.00116147
[200]	cv_agg's auc: 0.76238 + 0.00113755
[250]	cv_agg's auc: 0.767197 + 0.000981763
[300]	cv_agg's auc: 0.770956 + 0.000897454
[350]	cv_agg's auc: 0.773852 + 0.000912315
[400]	cv_agg's auc: 0.776179 + 0.000977148
[450]	cv_agg's auc: 0.778041 + 0.00109723
[500]	cv_agg's auc: 0.779664 + 0.00112941
[550]	cv_agg's auc: 0.78101 + 0.00124438
[600]	cv_agg's auc: 0.782161 + 0.00127704
[650]	cv_agg's auc: 0.783214 + 0.00134361
[700]	cv_agg's auc: 0.784082 + 0.00141449
[750]	cv_agg's auc: 0.784872 + 0.0014907
[800]	cv_agg's auc: 0.785539 + 0.00153992
[850]	cv_agg's auc: 0.78616 + 0.00161908
[900]	cv_agg's auc: 0.786697 + 0.00167577
[950]	cv_agg's auc: 0.787172 + 0.00168669
[1000]	cv_agg's auc: 0.78764 + 0.00172747
[1050]	cv_agg's auc: 0.787991 + 0.00176749
[1100]	cv_agg's auc

In [12]:
# execute model
print("Starting Light GBM Training. Train shape: {}, test shape: {}, best iter: {}"
      .format(train_df.shape, test_df.shape, len(models_cv["auc-mean"])))
models_train = lgb.train(param,dtrain,num_boost_round=len(models_cv["auc-mean"]))
# print("best model iteration: {}" .format(models_train.best_iteration)) # best iteration
# print("best model score: {}" .format(models_train.best_score)) # best score

Starting Light GBM Training. Train shape: (307511, 1309), test shape: (48744, 1309), best iter: 3486
best model iteration: 0
best model score: defaultdict(<class 'dict'>, {})


In [15]:
# predict target (model.best_iteration??)
print("Starting Light GBM Predict. Train shape: {}, test shape: {}, num iter: {}"
      .format(train_df.shape, test_df.shape, models_train.best_iteration))
pred_train = models_train.predict(X_train)

Starting Light GBM Predict. Train shape: (307511, 1309), test shape: (48744, 1309), num iter: 0


In [23]:
# roc_auc score ( 1に近いほどよい)
auc_mean = roc_auc_score(np.array(y_train),pred_train)
print("Light GBM {}-fold cross models_cvcvtion. loop: {}, auc_score:{}, cv score: {}"
      .format(NFOLD, i, auc_mean, models_cv["auc-mean"][-1]))

Light GBM 5-fold cross models_cvcvtion. loop: 1, auc_score:0.9021944332186309, cv score: 0.7930558603571797


In [24]:
# predict test data
pred_test = models_train.predict(X_test)

In [29]:
# make submit file
submit_file = pd.DataFrame({"SK_ID_CURR" : test_df["sk_id_curr"].astype("int"),"TARGET" : pred_test})

,SK_ID_CURR,TARGET
307511,100001,0.027910
307512,100005,0.123406
307513,100013,0.034792
307514,100028,0.047855
307515,100038,0.136660
307516,100042,0.035730
307517,100057,0.006980
307518,100065,0.022482
307519,100066,0.008184
307520,100067,0.079925


In [30]:
file_name = "light_gbm_test.csv"
submit_file.to_csv("../submit/" + file_name, index= False)

In [31]:
lgb.plot_importance(models_train, ax=ax, max_num_features=50, importance_type='split')
lgb.plot_importance(models_train, ax=ax1, max_num_features=50, importance_type='gain')
ax.set_title('Importance by splits')
ax1.set_title('Importance by gain')
plt.tight_layout()
plt.savefig('feature_importance.png')

<bound method Booster.dump_model of <lightgbm.basic.Booster object at 0x11b5cf320>>